# Ghouls Submissions pt.1
The first notebook was getting a bit crowded... so I wanted to try and put some models together here.

This notebook has my first 3 submissions to the Competition, Submission 3 yielded the best score on the leaderboard. The model parameters for submission 3 are from the data vizulization notebook. All models here are essentially gridsearch with cross validation in the GS.

In [5]:
import pandas as pd
import numpy as np

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [9]:
train.columns

Index(['id', 'bone_length', 'rotting_flesh', 'hair_length', 'has_soul',
       'color', 'type'],
      dtype='object')

In [11]:
# lets prep our features
dummies = pd.get_dummies(train['color'])
X = train.drop(['id','color','type'],axis=1)

# we also need to format our target variable 
type_dict = {'Ghoul':0,'Goblin':1,'Ghost':2}
Y = train.type.map(type_dict)

## First Submission

In [20]:
pipe_rf = Pipeline([('poly', PolynomialFeatures()),
                    ('clf', RandomForestClassifier(random_state=1))])

param_grid = [{'poly__degree':[2,3],
               'clf__criterion':['gini','entropy'],
               'clf__max_features':[1,2,3,4,5,6,7,8,"sqrt","log2"],
               'clf__min_samples_split':[1,2,3,4,5,6,7]}]

gs = GridSearchCV(estimator=pipe_rf,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 n_jobs=-1)

In [21]:
gs = gs.fit(X.values,Y.values)
print(gs.best_score_)
print(gs.best_params_)

0.754716981132
{'clf__criterion': 'entropy', 'clf__max_features': 4, 'poly__degree': 2, 'clf__min_samples_split': 1}


In [126]:
# or more simply
pipe_rf = Pipeline([('poly', PolynomialFeatures(degree=2)),
                    ('clf', RandomForestClassifier( criterion='entropy',
                                                    max_features=4,
                                                    min_samples_split=1,
                                                    random_state=1))])

scores = cross_val_score(pipe_rf,X.values,Y.values,cv=10)
print('CV Accuracy: {0:.3f} +/- {1:.3f}'.format(np.mean(scores),np.std(scores)))

CV Accuracy: 0.754 +/- 0.042


In [129]:
# now we can use the pipe above to generate our predicitons
pipe_rf.fit(X.values,Y.values)

Pipeline(steps=[('poly', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False))])

In [130]:
rf_pred = pipe_rf.predict(test.drop(['id','color'],axis=1))

In [131]:
rf_pred

array([0, 1, 0, 1, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1, 0, 2, 1, 2, 2, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 2, 0, 2, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 0, 2, 0, 1, 2, 1, 0, 0, 0, 0, 1,
       0, 1, 2, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 2, 2, 2, 1, 2, 2, 2, 2,
       1, 1, 2, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0, 2, 1, 2, 1, 0,
       1, 2, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 2, 0, 0, 2, 0, 2, 2, 1, 1, 0, 0, 1, 1, 2, 0, 0, 2, 0, 0,
       1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 2, 2, 0, 2,
       2, 1, 0, 0, 0, 2, 2, 0, 1, 2, 0, 1, 1, 0, 0, 0, 2, 2, 2, 2, 1, 2, 1,
       2, 1, 0, 1, 1, 2, 2, 1, 1, 0, 2, 0, 1, 1, 1, 0, 0, 0, 1, 2, 2, 1, 2,
       1, 0, 2, 2, 2, 0, 2, 0, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2,
       1, 0, 1, 2, 0, 2, 2, 2, 1, 2, 1, 1, 1, 0, 0, 1, 2, 0, 1, 2, 1, 0, 0,
       1, 0, 1, 2, 2, 1, 0, 0, 0, 1, 1, 0, 1, 2, 0, 2, 0, 1, 1, 1, 2, 1, 2,
       1, 1,

In [52]:
# setting up a model based on above to predict results - BASICALLY I DONT KNOW HOW TO USE THE PIPE TO PREDICT!
poly = PolynomialFeatures(degree=2)
clf = RandomForestClassifier( criterion='entropy',
                              max_features=4,
                              min_samples_split=1,
                              random_state=1)

clf.fit(poly.fit_transform(X.values),Y.values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

In [53]:
# a small transform of our test data such that we can make predicitons
test_ = poly.fit_transform(test.drop(['id','color'],axis=1))

# making the prediction
y_pred = clf.predict(test_)

In [70]:
# make submission
submission = pd.DataFrame({'id':test.id,'type':y_pred})
reverse_dict = {0:'Ghoul',1:'Goblin',2:'Ghost'}
submission['type'] = submission.type.map(reverse_dict)

submission.head()

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Goblin
4,13,Ghost


In [71]:
submission.to_csv('ghouls_01.csv',index=False)

## Second Submission

In [85]:
# the first day GS came up with this: {'clf__criterion': 'gini', 'clf__min_samples_split': 5, 'clf__max_features': 4}
clf = RandomForestClassifier(criterion='gini',min_samples_split=5,max_features=4)

x = pd.get_dummies(train.drop(['id','type'],axis=1))
y = train.type.map(type_dict)
clf.fit(x.values, y)

y_test = pd.get_dummies(test.drop(['id'],axis=1))
y_pred2 = clf.predict(y_test.values)

In [88]:
submission2 = pd.DataFrame({'id':test.id,'type':y_pred2})
reverse_dict = {0:'Ghoul',1:'Goblin',2:'Ghost'}
submission2['type'] = submission2.type.map(reverse_dict)

submission2.head()

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Ghost
4,13,Ghost


In [89]:
# before we submit lets just make sure this isnt the same submission 2x
submission.type == submission2.type

0       True
1       True
2       True
3      False
4       True
5       True
6       True
7      False
8       True
9       True
10      True
11      True
12     False
13     False
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21     False
22      True
23      True
24     False
25      True
26      True
27     False
28      True
29      True
       ...  
499     True
500     True
501     True
502     True
503    False
504     True
505     True
506     True
507     True
508     True
509     True
510    False
511     True
512     True
513     True
514     True
515     True
516     True
517     True
518     True
519     True
520     True
521     True
522     True
523     True
524     True
525     True
526     True
527     True
528     True
Name: type, dtype: bool

In [90]:
submission2.to_csv('ghouls_02.csv',index=False)

## Submission 3

In [91]:
# Fuck it lets try SVC {'clf__kernel': 'rbf', 'clf__gamma': 0.01, 'clf__C': 10.0}
from sklearn.svm import SVC

In [108]:
def make_poly(df,n):
    """make a poly and add back in dummies"""
    poly = PolynomialFeatures(degree=n)
    if 'type' in df.columns:
        df = df.drop(['type'],axis=1)
    train_poly = poly.fit_transform(df.drop(['id','color'],axis=1).values)
    dummies = pd.get_dummies(df.color)
    t = pd.DataFrame(train_poly)
    return pd.concat([t,dummies],axis=1)

In [111]:
clf = SVC(kernel='rbf',gamma=0.01,C=10,random_state=1)

clf.fit(make_poly(train,3).values, train.type.values)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.001, verbose=False)

In [112]:
y_pred3 = clf.predict(make_poly(test,3).values)

In [118]:
y_pred3[:10]

array(['Ghoul', 'Goblin', 'Ghoul', 'Ghost', 'Ghost', 'Ghost', 'Ghoul',
       'Ghoul', 'Goblin', 'Ghoul'], dtype=object)

In [119]:
submission3 = pd.DataFrame({'id':test.id,'type':y_pred3})

submission3.head()

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Ghost
4,13,Ghost


In [121]:
submission3.to_csv('ghouls_03.csv',index=False)